<a href="https://colab.research.google.com/github/adrichesa/MercadoLibre_PropertyFetcher/blob/main/adrianChesanouskiTrabajoIntegrador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Descripción general:
Este script realiza una solicitud a la API de FRED para obtener datos históricos del Índice de Precios al Consumidor (CPI)
de EE.UU., calcula la inflación mensual, almacena los datos transformados en formato Parquet y en una base de datos SQLite,
y finalmente imprime la inflación calculada para un público no especializado.

Paso 1: Importar Librerías
En la primera celda, importa todas las librerías necesarias para ello:

In [2]:
import pandas as pd
import requests
from sqlalchemy import create_engine


Paso 2: Configuración Inicial y Función de Extracción de Datos
En la segunda celda, puede configurar las variables iniciales, como la URL de la API y las funciones para extraer datos:

In [3]:
# Configuración de la API
api_key = '1436b6666daf413c6fbbafad7e8697b1'
series_id = 'CPIAUCSL'
base_url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json'

# Función para obtener datos de la API de FRED
def fetch_inflation_data():
    response = requests.get(base_url)
    if response.status_code == 200:
        return response.json()['observations']
    else:
        print(f"Error al acceder a la API de FRED: {response.status_code}")
        return None


Paso 3: Procesamiento de los Datos
En la tercera celda, realiza el procesamiento de los datos que se obtuvo de la API, como convertirlos en un DataFrame:

In [4]:
# Obtener los datos de la API y convertirlos en un DataFrame
observations = fetch_inflation_data()
df = pd.DataFrame(observations)

# Mostrar las primeras filas del DataFrame para verificar los datos
df.head()


,realtime_start,realtime_end,date,value
0,2024-08-14,2024-08-14,1947-01-01,21.48
1,2024-08-14,2024-08-14,1947-02-01,21.62
2,2024-08-14,2024-08-14,1947-03-01,22.0
3,2024-08-14,2024-08-14,1947-04-01,22.0
4,2024-08-14,2024-08-14,1947-05-01,21.95


Paso 4: Almacenamiento de Datos

En la cuarta celda, guarda los datos en Parquet y SQLite:

In [ ]:
# Guardar los datos en formato Parquet
df.to_parquet('fred_inflation_data.parquet', index=False)

# Guardar los datos en una base de datos SQLite
engine = create_engine('sqlite:///fred_inflation_data.db')
df.to_sql('inflation', con=engine, if_exists='replace', index=False)

print("Datos almacenados correctamente en Parquet y SQLite.")


Datos almacenados correctamente en Parquet y SQLite.


Paso 5: Transformación de Datos
En la quinta celda, realiza las transformaciones necesarias:

In [ ]:
# Convert the 'value' column to numeric type
df['value'] = pd.to_numeric(df['value'])

# Realizar transformaciones como manejar valores nulos y calcular variaciones porcentuales
df.fillna(method='ffill', inplace=True)
df['variacion_pct'] = df['value'].pct_change() * 100

# Mostrar el DataFrame con las transformaciones
df.head()

<ipython-input-5-45673fbe3dbc>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


,realtime_start,realtime_end,date,value,variacion_pct
0,2024-08-14,2024-08-14,1947-01-01,21.48,NaN
1,2024-08-14,2024-08-14,1947-02-01,21.62,0.651769
2,2024-08-14,2024-08-14,1947-03-01,22.00,1.757632
3,2024-08-14,2024-08-14,1947-04-01,22.00,0.000000
4,2024-08-14,2024-08-14,1947-05-01,21.95,-0.227273


Paso 6: Cálculo de la Inflación

En la sexta celda, calcula la inflación utilizando la fórmula correspondiente:

In [ ]:
# Calcular inflación utilizando los últimos dos valores
cpi_actual = df.iloc[-1]['value']
cpi_anterior = df.iloc[-2]['value']
inflacion = ((cpi_actual - cpi_anterior) / cpi_anterior) * 100

# Imprimir la fórmula y el resultado
print(f"Inflación = (({cpi_actual} - {cpi_anterior}) / {cpi_anterior}) x 100 ≈ {inflacion:.2f}%")


Inflación = ((313.534 - 313.049) / 313.049) x 100 ≈ 0.15%


Paso 7: Visualización de la Serie de Datos
Finalmente, en la séptima celda, puedes imprimir toda la serie de datos:

In [ ]:
# Imprimir la serie de datos
print(df[['date', 'value']])


           date    value
0    1947-01-01   21.480
1    1947-02-01   21.620
2    1947-03-01   22.000
3    1947-04-01   22.000
4    1947-05-01   21.950
..          ...      ...
926  2024-03-01  312.230
927  2024-04-01  313.207
928  2024-05-01  313.225
929  2024-06-01  313.049
930  2024-07-01  313.534

[931 rows x 2 columns]


Resultado Final
Siguiendo estos pasos, cada celda de la notebook se ejecutará de manera independiente, y podrá ver los resultados justo debajo de cada celda, lo que permitirá analizar los resultados paso a paso.